## 1. Импорт библиотек

In [1]:
import os

import pandas as pd

from utils.cleaning import normalize_street, normalize_district, drop_outlers
from utils.features import get_total_price

## 2. Чтение и подготовка данных

Задаём пути к файлам, параметры чтения CSV и загружаем данные с проверкой ошибок.

In [2]:
HOME = os.getcwd().removesuffix("/notebooks")

In [3]:
# Настраиваем словари с параметрами чтения CSV: путь, кодировка, дата-колонки
rides_options = {
    "filepath_or_buffer": f"{HOME}/data/rides.csv",
    "encoding": "utf-8",
    "parse_dates": ["Start Date", "End Date"],
}

weather_options = {
    "filepath_or_buffer": f"{HOME}/data/weather.csv",
    "encoding": "utf-8",
    "parse_dates": ["Datetime"]
}

try:
    rides_data = pd.read_csv(**rides_options)
    weather_data = pd.read_csv(**weather_options)
except pd.errors.ParserError as e:
    print(f"Error: {e}")
    rides_data = pd.read_csv(**rides_options, sep=";")
    weather_data = pd.read_csv(**weather_options, sep=";")

## 3. Переименование колонок

Приводим названия колонок к snake_case для удобства работы.

In [4]:
rides_data.rename(lambda col: col.strip().lower().replace(" ", "_"), axis="columns", inplace=True)

weather_data.rename(lambda col: col.strip().lower().replace(" ", "_"), axis="columns", inplace=True)

## 4. Очистка данных о поездках

### 4.1. Преобразование типов

Колонка `promo` содержит бинарные значения (1 — акция использована, 0 — нет).
Преобразуем её в категориальный признак и заменяем значения на "Да" / "Нет" для удобства анализа и визуализации.

In [21]:
rides_data["promo"] = rides_data["promo"].apply(lambda val: "Да" if val == 1 else "Нет")

,id,start_date,end_date,day_of_week,start_location,start_district,end_location,end_district,distance,promo,duration_minutes,total_price
0,100340,2023-04-22 09:01:03,2023-04-22 09:24:45,5,шевченко,юго-восточный,майская,октябрьский,4069.0,Нет,24,126
1,100341,2023-04-22 09:01:07,2023-04-22 09:29:32,5,чехова,заречный,спортивная,октябрьский,4515.0,Нет,28,142
2,100342,2023-04-22 09:04:27,2023-04-22 09:42:46,5,буденного,октябрьский,суворова,ленинский,4520.0,Нет,38,182
3,100343,2023-04-22 09:06:34,2023-04-22 09:38:23,5,труда,центральный,рабочая,северо-западный,3881.0,Нет,32,158
4,100344,2023-04-22 09:09:27,2023-04-22 09:40:07,5,первомайская,северо-западный,юбилейная,юго-восточный,4029.0,Нет,31,154


### 4.2. Удаление дубликатов

In [6]:
rides_data = rides_data.drop_duplicates()

rides_data.duplicated().sum()

np.int64(0)

### 4.3. Нормализация адресов и районов

Применяем функции из модуля `cleaning` для стандартизации названий улиц и районов.

In [7]:
rides_data["start_location"] = rides_data["start_location"].apply(normalize_street)
rides_data["end_location"] = rides_data["end_location"].apply(normalize_street)

In [8]:
rides_data["start_district"] = rides_data["start_district"].apply(normalize_district)
rides_data["end_district"] = rides_data["end_district"].apply(normalize_district)

### 4.4. Удаление пропусков
На первичном анализе было выявлено, что доля пропусков в данных менее 1%, поэтому их строки можно безопасно удалить.


In [9]:
rides_data = rides_data.dropna()

rides_data.isnull().sum()

id                0
start_date        0
end_date          0
start_location    0
start_district    0
end_location      0
end_district      0
distance          0
promo             0
dtype: int64

### 4.5. Удаление экстремальных значением

Применяем метод межквартильного размаха (IQR) для очистки датасета от экстремальных значений в колонке `distance`.

In [10]:
rides_data = drop_outlers(rides_data, "distance", k=3)
rides_data = rides_data[rides_data["distance"] >= 200]

### 4.6. Создание новых признаков

Добавляем итоговое время поездки.

In [11]:
td = rides_data["end_date"] - rides_data["start_date"]

rides_data["duration_minutes"] = (td.dt.total_seconds() / 60).round().astype(int)

Добавляем день недели и перемещаем колонку на удобную позицию.

In [12]:
rides_data["day_of_week"] = rides_data["start_date"].dt.dayofweek

col = rides_data.pop('day_of_week')
rides_data.insert(3, 'day_of_week', col)

Добавляем итоговую стоимость поездки.

In [13]:
rides_data["total_price"] = rides_data.apply(get_total_price, axis=1)

## 5. Очистка данных о погоде

### 5.1. Удаление ненужных столбцов

Удаляется колонка wind_gust, поскольку она дублирует информацию о порывах ветра.

In [14]:
weather_data = weather_data.drop("wind_gust", axis=1)

### 5.2. Удаление дубликатов

In [15]:
weather_data = weather_data.drop_duplicates()

weather_data.duplicated().sum()

np.int64(0)

### 5.3. Удаление первой служебной строки

Первая строка содержит единицы измерения (°C, km/h, mm и т.п.), которые не нужны для анализа, поэтому её удаляем.

In [16]:
weather_data = weather_data.iloc[1:].reset_index(drop=True)

### 5.4. Преобразование текстовых колонок в числовые

* Столбцы с типом object могут содержать текстовые значения.
* Используем pd.to_numeric(errors="coerce"), чтобы некорректные значения превратить в NaN.
* Это важно для дальнейшей интерполяции и работы с временным рядом.

In [17]:
cat_cols = weather_data.select_dtypes(include=[object]).columns
for cat in cat_cols:
    weather_data[cat] = pd.to_numeric(weather_data[cat], errors="coerce")

### 5.5. Удаление выбросов

Для признака wind_speed применяем межквартильный размах (IQR) с коэффициентом k = 3.

Это позволяет удалить аномально высокие значения скорости ветра
(например, недостоверные пики выше 25–27 м/с).

In [18]:
weather_data = drop_outlers(weather_data, "wind_speed", k=3)

### 5.6. Интерполяция пропущенных значений

In [19]:
# Устанавливаем datetime как индекс для интерполяции методом `time`
weather_data = weather_data.set_index('datetime')

# Выбираем столбцы для интерполяции
cols_to_interp = [
    'temperature', 'wind_speed',
    'sunshine_duration', 'cloud_cover_total'
]
# Заполняем пропуски интерполяцией по времени
weather_data[cols_to_interp] = weather_data[cols_to_interp].interpolate(method='time')

# Возвращаем datetime обратно как колонку
weather_data = weather_data.reset_index()

## 6. Сохраняем очищенные файлы

In [20]:
rides_data.to_csv(f"{HOME}/data/cleaned_rides.csv", index=False)
weather_data.to_csv(f"{HOME}/data/cleaned_weather.csv", index=False)